# 1.Limpieza y Transformación del Dataset de Calificaciones

Este notebook describe el proceso de limpieza y transformación del dataset de calificaciones.

El objetivo es garantizar que los datos sean **consistentes, estandarizados, anónimos y útiles** para el análisis posterior y la construcción de modelos de machine learning.

## 1.1. Definición de Variables - Dataset de Calificaciones

| Variable                  | Descripción                                                                 | Tipo de Dato         |
|---------------------------|-----------------------------------------------------------------------------|----------------------|
| `Sede`                    | Nombre de la sede o campus donde estudia el estudiante                     | `str` (texto)        |
| `Estudiante`              | Nombre completo del estudiante                                              | `str` (texto)        |
| `Documento de identidad`  | Identificador único del estudiante                                          | `str` (texto, hasheado) |
| `Grado`                   | Grado académico que cursa el estudiante (ej. 1, 2, 3...)                    | `int` (entero)       |
| `Grupo`                   | Letra o código del grupo del estudiante dentro del grado                   | `str` (texto)        |
| `Periodo`                 | Periodo académico (ej. I, II, III, IV)                                      | `str` (texto)        |
| `Año`                     | Año académico                                                               | `int` (entero)       |
| `Intensidad Horaria`      | Número de horas dedicadas a la asignatura                                   | `int` (entero)       |
| `Asignatura`              | Nombre de la materia evaluada (ej. Matemáticas, Inglés)                    | `str` (texto)        |
| `Cognitiva`               | Nota del componente cognitivo (conocimiento teórico)                        | `float` (decimal)    |
| `Procedimental`           | Nota del componente procedimental (habilidades prácticas)                   | `float` (decimal)    |
| `Actitudinal`             | Nota del componente actitudinal (comportamiento, participación)             | `float` (decimal)    |
| `Axiologica`              | Nota del componente axiológico (valores, ética)                             | `float` (decimal)    |
| `Docente`                 | Nombre del docente que reporta la calificación                              | `str` (texto)        |
| `Resultado`               | Nota final integrada de la asignatura                                       | `float` (decimal)    |
| `Nivel`                   | Nivel de desempeño asignado (ej. Bajo, Básico, Alto, Superior)              | `str` (texto)        |


In [1]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.transform.data_cleaning_notes import NotesDataCleaner
from scripts.utils.hash_utility import HashUtility

In [2]:
# comprobar que el metodo de limpieza existe
# y que se puede importar correctamente
from scripts.transform.data_cleaning_notes import NotesDataCleaner
print(dir(NotesDataCleaner))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'codificar_periodos', 'limpiar_columnas_numericas', 'load_and_clean_data']


## 1.2. Carga de dataset de Calificaciones

- Cargar fuente original de datos de calificaciones  
    - dataset de notas 2021-2022 limpio
    - dataset de notas 2023-2025 para limpiar


In [2]:
#hacer apend con las notas de los años anteriores

'''df_2021_2022 = pd.read_csv("../../data/raw/calificaciones-clean-2021-2022.csv")

df_2021_2022.info()
# Conteo de valores nulos
df_2021_2022.isnull().sum()
'''

'df_2021_2022 = pd.read_csv("../../data/raw/calificaciones-clean-2021-2022.csv")\n\ndf_2021_2022.info()\n# Conteo de valores nulos\ndf_2021_2022.isnull().sum()\n'

In [3]:
## consolidar csv guardados
carpeta_pdf = os.path.abspath('../../data/raw/calificaciones')
df = pd.DataFrame()
for root, _, files in os.walk(carpeta_pdf):
    for archivo in files:
        if archivo.endswith(".csv"):
            csv_path = os.path.join(root, archivo)
            df_temp = pd.read_csv(csv_path)
            df = pd.concat([df, df_temp], ignore_index=True)
        
df.to_csv(os.path.join(carpeta_pdf, 'calificaciones-raw-2023-2024-2025.csv'), index=False)


In [4]:
df_2023_2025 = pd.read_csv("../../data/raw/calificaciones/calificaciones-raw-2023-2024-2025.csv")

df_2023_2025.info()
# Conteo de valores nulos
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32676 entries, 0 to 32675
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sede                    32676 non-null  object 
 1   Estudiante              32676 non-null  object 
 2   Documento de identidad  32676 non-null  int64  
 3   Grado                   32676 non-null  int64  
 4   Grupo                   32676 non-null  object 
 5   Periodo                 32676 non-null  object 
 6   Año                     32676 non-null  int64  
 7   Intensidad Horaria      32676 non-null  int64  
 8   Asignatura              32676 non-null  object 
 9   Cognitiva               32571 non-null  float64
 10  Procedimental           32571 non-null  float64
 11  Actitudinal             32676 non-null  object 
 12  Axiologica              32676 non-null  int64  
 13  Docente                 32660 non-null  object 
 14  Resultado               32676 non-null

Sede                        0
Estudiante                  0
Documento de identidad      0
Grado                       0
Grupo                       0
Periodo                     0
Año                         0
Intensidad Horaria          0
Asignatura                  0
Cognitiva                 105
Procedimental             105
Actitudinal                 0
Axiologica                  0
Docente                    16
Resultado                   0
Nivel                       0
dtype: int64

## 1.3. Normalizar nombres de columnas

In [5]:
#imprimir nombres de columnas
print(df_2023_2025.columns)
##print(df_2021_2022.columns)

Index(['Sede', 'Estudiante', 'Documento de identidad', 'Grado', 'Grupo',
       'Periodo', 'Año', 'Intensidad Horaria', 'Asignatura', 'Cognitiva',
       'Procedimental', 'Actitudinal', 'Axiologica', 'Docente', 'Resultado',
       'Nivel'],
      dtype='object')


### - Eliminar caracteres especiales y letras de columna


In [6]:
cleaner = NotesDataCleaner(df_2023_2025)

In [7]:
columnas_a_limpieza = [ "Actitudinal"]
df_2023_2025 = cleaner.limpiar_columnas_numericas( columnas_a_limpieza)

Valores modificados por columna:
- Actitudinal: 105 valores corregidos


In [8]:
df_2023_2025.info()
# Conteo de valores nulos
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32676 entries, 0 to 32675
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sede                    32676 non-null  object 
 1   Estudiante              32676 non-null  object 
 2   Documento de identidad  32676 non-null  int64  
 3   Grado                   32676 non-null  int64  
 4   Grupo                   32676 non-null  object 
 5   Periodo                 32676 non-null  object 
 6   Año                     32676 non-null  int64  
 7   Intensidad Horaria      32676 non-null  int64  
 8   Asignatura              32676 non-null  object 
 9   Cognitiva               32571 non-null  float64
 10  Procedimental           32571 non-null  float64
 11  Actitudinal             32676 non-null  int64  
 12  Axiologica              32676 non-null  int64  
 13  Docente                 32660 non-null  object 
 14  Resultado               32676 non-null

Sede                        0
Estudiante                  0
Documento de identidad      0
Grado                       0
Grupo                       0
Periodo                     0
Año                         0
Intensidad Horaria          0
Asignatura                  0
Cognitiva                 105
Procedimental             105
Actitudinal                 0
Axiologica                  0
Docente                    16
Resultado                   0
Nivel                       0
dtype: int64

In [10]:
#cambiar el nombre de la columna "id" a "id_calificacion"
#df_2023_2025.rename(columns={"Resultado": "Prom"}, inplace=True)

df_2023_2025.rename(columns={"Cognitiva": "Cog"}, inplace=True)
df_2023_2025.rename(columns={"Procedimental": "Proc"}, inplace=True)
df_2023_2025.rename(columns={"Actitudinal": "Act"}, inplace=True)
df_2023_2025.rename(columns={"Axiologica": "Axi"}, inplace=True)


In [11]:
df_2023_2025.info()
# Conteo de valores nulos       
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32676 entries, 0 to 32675
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sede                    32676 non-null  object 
 1   Estudiante              32676 non-null  object 
 2   Documento de identidad  32676 non-null  int64  
 3   Grado                   32676 non-null  int64  
 4   Grupo                   32676 non-null  object 
 5   Periodo                 32676 non-null  object 
 6   Año                     32676 non-null  int64  
 7   Intensidad Horaria      32676 non-null  int64  
 8   Asignatura              32676 non-null  object 
 9   Cog                     32571 non-null  float64
 10  Proc                    32571 non-null  float64
 11  Act                     32676 non-null  int64  
 12  Axi                     32676 non-null  int64  
 13  Docente                 32660 non-null  object 
 14  Resultado               32676 non-null

Sede                        0
Estudiante                  0
Documento de identidad      0
Grado                       0
Grupo                       0
Periodo                     0
Año                         0
Intensidad Horaria          0
Asignatura                  0
Cog                       105
Proc                      105
Act                         0
Axi                         0
Docente                    16
Resultado                   0
Nivel                       0
dtype: int64

## 1.4. Limpieza de los datos
Realiza limpieza básica solo a df_2023_2025 ya que :ese proceso se corrio previamente para df_2021_2022
- Quita espacios de nombres de columnas  
- Convierte columnas de notas a numérico  
- Convierte texto a mayúsculas  

In [12]:

df_2023_2025 = cleaner.load_and_clean_data()
df_2023_2025.info()
df_2023_2025.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32676 entries, 0 to 32675
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sede                    32676 non-null  object 
 1   Estudiante              32676 non-null  object 
 2   Documento_de_identidad  32676 non-null  int64  
 3   Grado                   32676 non-null  object 
 4   Grupo                   32676 non-null  object 
 5   Periodo                 32676 non-null  object 
 6   Año                     32676 non-null  object 
 7   Intensidad_Horaria      32676 non-null  int64  
 8   Asignatura              32676 non-null  object 
 9   Cog                     32571 non-null  float64
 10  Proc                    32571 non-null  float64
 11  Act                     32676 non-null  int64  
 12  Axi                     32676 non-null  int64  
 13  Docente                 32676 non-null  object 
 14  Resultado               32676 non-null

Sede                        0
Estudiante                  0
Documento_de_identidad      0
Grado                       0
Grupo                       0
Periodo                     0
Año                         0
Intensidad_Horaria          0
Asignatura                  0
Cog                       105
Proc                      105
Act                         0
Axi                         0
Docente                     0
Resultado                   0
Nivel                       0
dtype: int64

## 1.5. Anonimización de los datos

Se aplicó **hashing seguro** a las variables sensibles:

- `id` se reemplaza con hashing 
- `Estudiante` (identificador único basado en nombre y sede) y aplica hashing

Estas columnas fueron eliminadas posteriormente para preservar la privacidad de los estudiantes.

- Se aplica funcion para codificar periodos

In [ ]:
# Aplicar hashing al documento de identificación y eliminar la columna original
df_2023_2025["Identificación"] = df_2023_2025["Documento_de_identidad"].apply(HashUtility.hash_stable)
df_2023_2025.drop(columns=["Documento_de_identidad"], inplace=True)
# Aplicar hashing a los nombres y apellidos y eliminar la columna original
df_2023_2025["Estudiante"] = df_2023_2025["Estudiante"].replace(" ", "")+df_2023_2025["Sede"].astype(str)
df_2023_2025["Estudiante"] = df_2023_2025["Estudiante"].apply(HashUtility.hash_stable)
# Mapear Docentes
df['Docente'] = df['Docente'].replace('INVESTIGACIÓNNICOLÁS CONTRERAS ANAVE', 'NICOLÁS CONTRERAS ANAVE')



In [14]:
df_2023_2025= cleaner.codificar_periodos(df_2023_2025) 

## 1.6. Unificar todos los dataframes de notas

In [13]:
# Concatena los DataFrames
## df_append = pd.concat([df_2023_2025, df_2021_2022], ignore_index=True)


In [14]:
## df_append.info()
# Conteo de valores nulos
## df_append.isnull().sum()

In [15]:
##BORRAR DUPLICADOS
duplicados = df_2023_2025[df_2023_2025.duplicated(subset=["Estudiante", "Identificación", "Grado", "Grupo", "Periodo", "Año", "Asignatura"], keep=False)]
df_2023_2025.drop_duplicates(subset=["Estudiante", "Identificación", "Grado", "Grupo", "Periodo", "Año", "Asignatura"], keep='first', inplace=True)
duplicados.info()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sede                0 non-null      object 
 1   Estudiante          0 non-null      object 
 2   Grado               0 non-null      object 
 3   Grupo               0 non-null      object 
 4   Periodo             0 non-null      object 
 5   Año                 0 non-null      object 
 6   Intensidad_Horaria  0 non-null      int64  
 7   Asignatura          0 non-null      object 
 8   Cog                 0 non-null      float64
 9   Proc                0 non-null      float64
 10  Act                 0 non-null      int64  
 11  Axi                 0 non-null      int64  
 12  Docente             0 non-null      object 
 13  Resultado           0 non-null      int64  
 14  Nivel               0 non-null      object 
 15  Identificación      0 non-null      object 
dtypes: float64(2), int64(4), 

In [16]:
df_2023_2025.info()
# Conteo de valores nulos
df_2023_2025.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32676 entries, 0 to 32675
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sede                32676 non-null  object 
 1   Estudiante          32676 non-null  object 
 2   Grado               32676 non-null  object 
 3   Grupo               32676 non-null  object 
 4   Periodo             32676 non-null  object 
 5   Año                 32676 non-null  object 
 6   Intensidad_Horaria  32676 non-null  int64  
 7   Asignatura          32676 non-null  object 
 8   Cog                 32571 non-null  float64
 9   Proc                32571 non-null  float64
 10  Act                 32676 non-null  int64  
 11  Axi                 32676 non-null  int64  
 12  Docente             32676 non-null  object 
 13  Resultado           32676 non-null  int64  
 14  Nivel               32676 non-null  object 
 15  Identificación      32676 non-null  object 
dtypes: f

Sede                    0
Estudiante              0
Grado                   0
Grupo                   0
Periodo                 0
Año                     0
Intensidad_Horaria      0
Asignatura              0
Cog                   105
Proc                  105
Act                     0
Axi                     0
Docente                 0
Resultado               0
Nivel                   0
Identificación          0
dtype: int64

In [17]:
df_2023_2025.sample(5)

,Sede,Estudiante,Grado,Grupo,Periodo,Año,Intensidad_Horaria,Asignatura,Cog,Proc,Act,Axi,Docente,Resultado,Nivel,Identificación
5321,FUSAGASUGÁ,8cf0fb07aadef5177adbf880eb6ca8e54788743edbd887...,2,A,1,2023,1,ARTES,84.0,84.0,90,80,SANDRA SANTISTEBAN OSTOS,85,ALTO,82958b72de4fa0a2367c63e25a7c3786fd4d6370678f47...
16988,FUSAGASUGÁ,72f8badcfeab59cc212a41ceb4ea6b843f552c035b3008...,1,A,2,2024,3,CIENCIAS SOCIALES,93.0,93.0,93,94,JIMENA GÓMEZ ARÉVALO,93,ALTO,4f5975d3d87715276f1500db20f5febb50583de862b26a...
25737,GIRARDOT,9c0704dcdaea28fd7fd4e92c792f09a5147d6bdd6a9cd0...,5,A,2,2024,3,LENGUA CASTELLANA,77.0,78.0,87,87,VALENTINA SARABIA VARGAS,80,ALTO,d529e142c0a86bc3311b2efcb6206465530d74aa80de42...
26876,GIRARDOT,8e59eb14e486acf1c624acc4aea530c7007d4d81f1967c...,3,A,1,2024,3,LENGUA CASTELLANA,74.0,73.0,90,96,PILAR RODRÍGUEZ ISAZA,80,ALTO,65f0e53fa2b06691cacb4361307df1bbd242e9c0fe98bb...
30007,FUSAGASUGÁ,e88b9c444406cb0e41d00ef3cd6de1379ab094434d396c...,3,A,1,2025,3,EDUCACIÓN FÍSICA,94.0,94.0,95,98,STEVEN BARRERO BUITRAGO,94,ALTO,b6f6f0de1cb6006c0c5ab9b2dc5c22cdd1a44ea999b8e2...


In [18]:
#reemplace nombres de valores de asignaturas
df_2023_2025["Asignatura"] = df_2023_2025["Asignatura"].replace({
    "ENGLISH": "INGLÉS",
    "FRANÇAIS": "FRANCÉS"})


In [19]:
df
df_2023_2025.isnull().sum()

Sede                    0
Estudiante              0
Grado                   0
Grupo                   0
Periodo                 0
Año                     0
Intensidad_Horaria      0
Asignatura              0
Cog                   105
Proc                  105
Act                     0
Axi                     0
Docente                 0
Resultado               0
Nivel                   0
Identificación          0
dtype: int64

In [20]:
df_2023_2025.to_csv("../../data/processed/data_clean_notes_2023_2025.csv", index=False)